Hi,
so this is the first thing that comes to my mind:

1. $n$ agents, $m$ projects
1. in iterations, add "chaos":
  1. randomly choose a project $p$ and a random cycle $C$ on a set of some $z$ agents that are not yet doing anything with this project $p$ **and at least one other project** (can be different for each voter in $C$);
  1. for each agent in this cycle, choose a superset of projects $S$, $p \in S$, to delegate to the next agent on the cycle (e.g., if we choose project $p_1$ and agents $a, b, c$, we can delegate $\{p_1,p_2\}$ from $a$ to $b$; delegate $\{p_1, p_3, p_{65}\}$ from $b$ to $c$; and delegate $\{p_1, p_6\}$ from $c$ to $a$)
  1. halt whenever you cannot find more chaos to add = there is no project $p$ which is non-delegated by $\geq 2$ agents s.t. each agent has at least one other non-delegated project.
1. the variables are budgets for the parts of the partition




In [1]:
import random, itertools, os
from collections import defaultdict
from frozendict import frozendict
import pyomo.environ as pyo
#os.environ["PATH"] +=":/home/martin/BIG/gurobi912/linux64/bin/"
#os.environ["GUROBI_HOME"] = "/home/martin/BIG/gurobi912/linux64/"

In [2]:
n = 10 # number of voters
m = 5 # number of projects

In [3]:
def create(n,m):
    voters = defaultdict(dict)
    # voters[i][S] = some voter v' to whom i delegates S
    voters_unassigned = {v:set(range(m)) for v in range(n)}
    # voters_unassigned[i] = set of projects that i does not yet assign to anyone
    projects = list(range(m))
    projects_to_assign = {p:set(range(n)) for p in range(m)}
    # project_to_assign[p] = set of voters who do not yet delegate p
    
    while True:
        random.shuffle(projects)
        for p in projects:
            found = False
            if len(projects_to_assign[p]) == 1:
                # only one voter doesn't do anything with p
                # delegate it to himself
                v = projects_to_assign[p].pop()
                voters[v][frozenset([p])] = v
                voters_unassigned[v].remove(p)
                continue
            feasible_voters = [v for v in projects_to_assign[p] if len(voters_unassigned[v]) > 1]
            if len(feasible_voters) < 2:
                # no way to put the available voters in a non-trivial cycle
                continue
            found = True
            break
        if not found:
            break
            
        #print("feasible_voters:", feasible_voters)
        z = random.randint(2,len(feasible_voters))
        C = random.sample(feasible_voters,z)
        #print("C:",C)
        for (i,v) in enumerate(C):
            s = random.randint(1,len(voters_unassigned[v])-1)
            S = set(random.sample(list((pp for pp in voters_unassigned[v] if pp != p)),s))
            #print("S:",S,"list(voters_unassigned[v]):",list(voters_unassigned[v]))
            S.add(p)
            voters[v][frozenset(S)] = C[(i+1)%z]
            voters_unassigned[v] -= S
            #print("S:",S, "v:",v,"p:",p,"s:", s)
            for pp in S:
                #print("pp:",pp,"projects_to_assign[pp]",projects_to_assign[pp])
                #print("voters[v]",voters[v])
                projects_to_assign[pp].remove(v)
    
    # Assign remaining non-assigned project to one-self (no delegation)
    for v in range(n):
        for p in list(voters_unassigned[v]):
                #v = projects_to_assign[p].pop()
                voters[v][frozenset([p])] = v
                voters_unassigned[v].remove(p)
                projects_to_assign[p].remove(v)
    
    #print("projects_to_assign:",projects_to_assign)
    #print("voters_unassigned:", voters_unassigned)
    
    return voters
        
    #random.shuffle
    #random.randint(a,b)
    

The result of the above function is a dictionary `res` indexed by the voters, such that for a voter $v$, `res[v]` is the partition of projects, e.g. `res[v][S]=d` if $v$ delegates $S \subseteq [m]$ to $d \in [n]$, and if $d=v$ then it means that $v$ is delegating this to himself. 

The optimization problem is to find an "initial solution" $(x_{ij})_{(i,j) \in [n] \times [m]}$ such that $\|x - f(x)\| \leq \|f(x) - f(f(x))\|$ under some norm, which would prove that $f$ is not contracting for this instance. $f$ is defined to exactly preserve the solution for self-delegated bundles (i.e. those $x$'s are constants) and to satisfy some notion of proportionality for the rest.

In [4]:
res = create(n,m)
#print(res)
# Verification that we really did delegate everything etc.
for (p,pp) in itertools.combinations(range(m),2):
    vs = []
    for v in range(n):
        SS = set()
        for S in res[v].keys():
            SS = SS.union(S)
        if (p not in SS) and (pp not in SS):
            vs.append(v)
    #vs = [v for v in res if (p not in set().union(res[v].keys())) and (pp not in set().union(res[v].keys()))]
    if len(vs) > 1:
        print (p,pp)

for v in res:
    for S in res[v]:
        if len(S) == 1:
            if res[v][S] != v:
                print(v,S)

In [5]:
res[4]

{frozenset({0, 1, 2, 3, 4}): 8}

## The Model

In [14]:
def contraction_counterexample_model(res, n,m,fix_weights=False, even_split=False,proportionality="martin",horizon=3):
    model = pyo.ConcreteModel()

    model.three = pyo.RangeSet(1,horizon)
    model.two = pyo.RangeSet(1,horizon-1)
    model.I = pyo.RangeSet(0, n-1)
    model.J = pyo.RangeSet(0, m-1)

    model.x = pyo.Var(model.three, model.I, model.J, domain=pyo.UnitInterval)
    model.d = pyo.Var(model.I, model.J, domain=pyo.UnitInterval) # defaults

    model.D = pyo.Set(initialize=list((v,S) for v in res for S in res[v])) # delegated sets
    model.DD = pyo.Set(initialize=list(((v,S,p) for v in res for S in res[v] for p in S))) # delegated sets \times projects
    model.DDD = pyo.Set(initialize=list(((i,v,S,p) for i in model.two for v in res for S in res[v] for p in S))) # delegated horizon-1 \ times sets \times projects
    model.b = pyo.Var(model.D, domain=pyo.UnitInterval) # budget variables
    if not fix_weights:
        model.w = pyo.Var(model.D, domain=pyo.NonNegativeReals, bounds=(1,5), initialize=10) # delegation weights variables
        #model.w = pyo.Var(model.D, domain=pyo.Integers, bounds=(1,5)) # delegation weights variables
    else:
        model.w = {(v,S): 10 for v in res for S in res[v]}
    
    def x_budget_constraint_rule(m,v,S):
        return sum(model.x[(1,v,j)] for j in S) == model.b[(v,S)]

    model.x_budget_constraints = pyo.Constraint(model.D, rule=x_budget_constraint_rule)

    def defaults_constraint_rule(m,v,S):
        return sum(model.d[(v,j)] for j in S) == model.b[(v,S)]
    
    def defaults_constraint_even_split_rule(m,v,S,p):
        return model.d[(v,p)] == model.b[(v,S)] / len(S)

    if even_split:
        model.defaults_constraints = pyo.Constraint(model.DD, rule=defaults_constraint_even_split_rule)
    else:
        model.defaults_constraints = pyo.Constraint(model.D, rule=defaults_constraint_rule)

    def budget_simplex_constraint_rule(m,v):
        return sum(model.b[(v,S)] for S in res[v]) == 1

    model.budget_simplex_constraints = pyo.Constraint(model.I, rule=budget_simplex_constraint_rule)
    # model.b[(v,S)] = how much budget we give delegation S
    # model.w[(v,S)] = how much weight we give the delegate


    # m is the model, always an implicit argument
    def simplex_rule(m, i):
        return sum(m.x[(1,i,j)] for j in m.J) == 1

    # the next line creates one constraint for each member of the set model.I
    # sum of contributions of one voter over all projects is 1
    model.simplex_constraints = pyo.Constraint(model.I, rule=simplex_rule)
    # Proportionality constraints

    def martin_proportionality_rule(m,it,v,S,p):
        # m is model, v voter, S delegated subset, it=1 or 2 is iteration
        delegate = res[v][S]
        denominator = sum(m.d[(v,p)] + m.w[(v,S)]*m.x[(it,delegate,p)] for p in S)
        return m.x[(it+1,v,p)] == ((m.d[(v,p)] + m.w[(v,S)]*m.x[(it,delegate,p)]) / denominator) * m.b[(v,S)]
    
    def original_proportionality_rule(m,it,v,S,p):
        # m is model, v voter, S delegated subset, it=1 or 2 is iteration
        delegate = res[v][S]
        denominator = sum(m.x[(it,delegate,p)] for p in S)
        return m.x[(it+1,v,p)] == ((m.x[(it,delegate,p)]) / denominator) * m.b[(v,S)]
    
    if proportionality == "martin":
        proportionality_rule = martin_proportionality_rule
    elif proportionality == "original":
        proportionality_rule = original_proportionality_rule

    def first_prop_rule(m,v,S,p):
        return proportionality_rule(m,1,v,S,p)

    def second_prop_rule(m,v,S,p):
        return proportionality_rule(m,2,v,S,p)
    
    #model.first_proportionality = pyo.Constraint(model.DD, rule=first_prop_rule)
    #model.second_proportionality = pyo.Constraint(model.DD, rule=second_prop_rule)
    model.proportionality = pyo.Constraint(model.DDD, rule=proportionality_rule)
    
    def initial_constraint_defaults_rule(m,v,p):
        return model.x[(1,v,p)] == model.d[(v,p)]
    
    #model.initial_defaults = pyo.Constraint(model.I, model.J, rule=initial_constraint_defaults_rule)
    
    def initial_constraint_evensplit_rule(m,v,S,p):
        # TODO must fix - to define so it works even if d is not even split
        return model.x[(1,v,p)] == model.b[(v,S)] / len(S)
    
    model.initial_evensplit = pyo.Constraint(model.DD, rule=initial_constraint_evensplit_rule)
    
    def objective_rule(m):
        # L1 norm
        # Horizon in the sense of dist(x,f^k(x)) < dist(f(x), f^{k+1}(x))
        #Delta1 = sum(abs(m.x[(1,i,j)] - m.x[(horizon-1,i,j)]) for i in m.I for j in m.J)
        #Delta2 = sum(abs(m.x[(2,i,j)] - m.x[(horizon,i,j)]) for i in m.I for j in m.J)
        # Horizon in the sense of dist(x, f(x)) < dist(f^k(x), f^{k+1}(x))
        Delta1 = sum(abs(m.x[(horizon-2,i,j)] - m.x[(horizon-1,i,j)]) for i in m.I for j in m.J)
        Delta2 = sum(abs(m.x[(horizon-1,i,j)] - m.x[(horizon,i,j)]) for i in m.I for j in m.J)
        # L2
        #Delta1 = sum((m.x[(1,i,j)] - m.x[(2,i,j)])**2 for i in m.I for j in m.J)
        #Delta2 = sum((m.x[(2,i,j)] - m.x[(3,i,j)])**2 for i in m.I for j in m.J)
        return Delta2 - Delta1
    model.OBJ = pyo.Objective(rule=objective_rule,sense=pyo.maximize)
    
    return model

In [15]:
n=10
m=5
res = create(n,m)

In [19]:
for i in range(100):
    res = create(n,m)
    model = contraction_counterexample_model(res, n,m,fix_weights=True, even_split=False,proportionality="martin",horizon=3)
    
    opt = pyo.SolverFactory("ipopt")
    %time results = opt.solve(model)
    #%time results = opt.solve(model, iterations=-1, HCS_max_iterations=10, HCS_tolerance=0.01, stopping_delta=0.99, stopping_mass=0.99, strategy="rand")
    #%time results = opt.solve(model, iterations=10, strategy="midpoint_guess_and_bound")
    #%time results = opt.solve(model, iterations=5, strategy="rand_guess_and_bound")
    #%time results = opt.solve(model, iterations=10, strategy="rand_distributed")
    #%time results = opt.solve(model, iterations=10, strategy="midpoint")
    #results.write()
    try: obj = pyo.value(model.OBJ)
    except ValueError: continue
    print()
    print("*************************")
    print("obj:", obj)
    print("*************************")
    if obj > 0.1:
        print("*************************")
        print("Found it!")
        print("*************************")
        break

    model.name="unknown";
      - termination condition: maxIterations
      - message from solver: Ipopt 3.14.4\x3a Maximum Number of Iterations
        Exceeded.
CPU times: user 208 ms, sys: 54 ms, total: 262 ms
Wall time: 4.23 s

*************************
obj: -0.038067886745269286
*************************
    model.name="unknown";
      - termination condition: maxIterations
      - message from solver: Ipopt 3.14.4\x3a Maximum Number of Iterations
        Exceeded.
CPU times: user 199 ms, sys: 59.6 ms, total: 258 ms
Wall time: 4.05 s

*************************
obj: -0.27782714970077044
*************************
    model.name="unknown";
      - termination condition: maxIterations
      - message from solver: Ipopt 3.14.4\x3a Maximum Number of Iterations
        Exceeded.
CPU times: user 202 ms, sys: 79.6 ms, total: 282 ms
Wall time: 6.65 s

*************************
obj: 0.3697859583064824
*************************
*************************
Found it!
*************************


In [21]:
# Weights are all 10
# Default are all evensplit
import numpy as np
b = dict()
for v in res:
    for S in res[v]:
        b[(v,S)] = model.b[(v,S)].value
# x is fixedpoint, y is some solution
x = np.zeros((n,m))
for i in model.I:
    for j in model.J:
        x[i,j] = model.x[(1,i,j)].value
counterexample = {"res": res, "x": x,"b":b}

In [22]:
#import cloudpickle as pickle
import pickle
name="n_10_m_5_contraction_fixedweights"
backup = open(name+".pickle","wb")
#model_file = open(name+"_model.pickle","wb")
pickle.dump(counterexample,backup)
backup.close()

In [23]:
res

defaultdict(dict,
            {5: {frozenset({0, 1, 3, 4}): 9, frozenset({2}): 5},
             9: {frozenset({0, 2, 3, 4}): 0, frozenset({1}): 9},
             0: {frozenset({0, 1, 2, 3, 4}): 8},
             8: {frozenset({0, 1, 2, 3, 4}): 3},
             3: {frozenset({1, 3, 4}): 2, frozenset({0, 2}): 4},
             2: {frozenset({1, 4}): 7,
              frozenset({0, 3}): 7,
              frozenset({2}): 2},
             7: {frozenset({2, 4}): 5,
              frozenset({0, 3}): 2,
              frozenset({1}): 7},
             1: {frozenset({0, 1, 2, 4}): 6, frozenset({3}): 1},
             6: {frozenset({0, 1, 2, 3}): 1, frozenset({4}): 6},
             4: {frozenset({0, 2, 3, 4}): 3, frozenset({1}): 4}})

In [24]:
b

{(5, frozenset({0, 1, 3, 4})): 0.7848317687777767,
 (5, frozenset({2})): 0.2151682308692209,
 (9, frozenset({0, 2, 3, 4})): 0.7981080790811242,
 (9, frozenset({1})): 0.2018919209257236,
 (0, frozenset({0, 1, 2, 3, 4})): 0.99999998942934,
 (8, frozenset({0, 1, 2, 3, 4})): 0.9999999894292197,
 (3, frozenset({1, 3, 4})): 0.6478262864823251,
 (3, frozenset({0, 2})): 0.352173713467751,
 (2, frozenset({1, 4})): 0.7473931310002383,
 (2, frozenset({0, 3})): 0.05137731373621123,
 (2, frozenset({2})): 0.20122955502853174,
 (7, frozenset({2, 4})): 0.49758406472008726,
 (7, frozenset({0, 3})): 0.25618809156829514,
 (7, frozenset({1})): 0.24622784390457378,
 (1, frozenset({0, 1, 2, 4})): 0.79553846092885,
 (1, frozenset({3})): 0.20446153893521685,
 (6, frozenset({0, 1, 2, 3})): 0.7955384609294323,
 (6, frozenset({4})): 0.20446153893463323,
 (4, frozenset({0, 2, 3, 4})): 0.9574707617030919,
 (4, frozenset({1})): 0.04252923822653633}

In [25]:
x

array([[0.2       , 0.2       , 0.2       , 0.2       , 0.2       ],
       [0.19888461, 0.19888461, 0.19888461, 0.20446154, 0.19888462],
       [0.02568866, 0.37369657, 0.20122956, 0.02568865, 0.37369657],
       [0.17608686, 0.2159421 , 0.17608686, 0.2159421 , 0.2159421 ],
       [0.23936769, 0.04252924, 0.23936769, 0.23936769, 0.23936769],
       [0.19620794, 0.19620794, 0.21516823, 0.19620794, 0.19620794],
       [0.19888461, 0.19888461, 0.19888461, 0.19888462, 0.20446154],
       [0.12809405, 0.24622784, 0.24879203, 0.12809405, 0.24879203],
       [0.2       , 0.2       , 0.2       , 0.2       , 0.2       ],
       [0.19952702, 0.20189192, 0.19952702, 0.19952702, 0.19952702]])